In [21]:
import time
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break








# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetworkArch1(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
        
# Define model
class NeuralNetworkArch2(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU(),
            nn.Linear(10, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

# Define model
class NeuralNetworkArch3(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


# Define model
class NeuralNetworkArch4(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits



NNs = [NeuralNetworkArch1(), NeuralNetworkArch2(), NeuralNetworkArch3(), NeuralNetworkArch4()]
NNNames = ['Arch1', 'Arch2', 'Arch3', 'Arch4']


def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
Using cpu device


In [22]:
traintimeNNs = [0, 0, 0, 0]
epochs = 5

for i, (NN, name) in enumerate(zip(NNs, NNNames)):
    print(name)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(NN.parameters(), lr=1e-3)
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        
        start_epoch = time.time()
        train(train_dataloader, NN, loss_fn, optimizer)
        if device == 'cuda':
            torch.cuda.synchronize()
        end_epoch = time.time()
        elapsed = end_epoch - start_epoch
        
        traintimeNNs[i] += elapsed
    print()


Arch1
Epoch 1
-------------------------------
loss: 2.301665  [    0/60000]
loss: 2.290602  [ 6400/60000]
loss: 2.281569  [12800/60000]
loss: 2.272102  [19200/60000]
loss: 2.266146  [25600/60000]
loss: 2.230227  [32000/60000]
loss: 2.239915  [38400/60000]
loss: 2.203684  [44800/60000]
loss: 2.199800  [51200/60000]
loss: 2.168964  [57600/60000]
Epoch 2
-------------------------------
loss: 2.172296  [    0/60000]
loss: 2.166104  [ 6400/60000]
loss: 2.119619  [12800/60000]
loss: 2.129073  [19200/60000]
loss: 2.095441  [25600/60000]
loss: 2.026235  [32000/60000]
loss: 2.056657  [38400/60000]
loss: 1.976640  [44800/60000]
loss: 1.978179  [51200/60000]
loss: 1.906451  [57600/60000]
Epoch 3
-------------------------------
loss: 1.935038  [    0/60000]
loss: 1.909572  [ 6400/60000]
loss: 1.803801  [12800/60000]
loss: 1.834811  [19200/60000]
loss: 1.744630  [25600/60000]
loss: 1.677772  [32000/60000]
loss: 1.706065  [38400/60000]
loss: 1.598629  [44800/60000]
loss: 1.623163  [51200/60000]
loss

In [23]:
for traintime, name in zip(traintimeNNs, NNNames):
    print(name, traintime)

Arch1 72.15912103652954
Arch2 70.25533246994019
Arch3 57.29960489273071
Arch4 79.83308053016663


In [25]:
def test_MCR(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        i = 0
        for X, y in test_dataloader:
            if i > 99:
                break
            i += len(X)
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    return (100 - correct) / 100


for i, (NN, name) in enumerate(zip(NNs, NNNames)):
    print(name + ' - MCR: ' + str(test_MCR(test_dataloader, NN, loss_fn)))

Arch1 - MCR: 0.14
Arch2 - MCR: 0.5
Arch3 - MCR: 0.09
Arch4 - MCR: 0.36
